# Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from contextlib import redirect_stdout
import numpy as np
import os
import pandas as pd
import plotly.express as px

from cost import *
from plotting import *
from prices import *
from imputation import *
from inflation import *
from regression import *
from utils import *

In [3]:
estimation_method = 'hardware-capex-opex'
compute_threshold_method = 'top_n'  # top_n, window_percentile
compute_threshold = 10  # e.g. 10 to select top 10; 75 to select top 25%
variant = 'depreciation-update'

estimation_method_lookup = {
    'cloud': estimate_cloud_costs,
    'amortized': estimate_amortized_hardware_costs,
    'hardware-acquisition': estimate_hardware_acquisition_cost,
    'hardware-capex-opex': estimate_hardware_capex_opex,
}
cost_estimation_function = estimation_method_lookup[estimation_method]

results_dir = f'results/{estimation_method}-{compute_threshold_method}={compute_threshold}-{variant}/'
os.makedirs(results_dir, exist_ok=True)

# Load data

In [4]:
frontier_pcd_df, hardware_df, price_df = load_data_for_cost_estimation(
    compute_threshold_method=compute_threshold_method, compute_threshold=compute_threshold,
)

In [5]:
len(frontier_pcd_df), len(hardware_df), len(price_df)

(67, 5510, 142)

# Cost estimation

In [6]:
with open(f'{results_dir}/cost_estimation.out', 'w') as f:
    with redirect_stdout(f):
        cost_df = cost_estimation_function(frontier_pcd_df, hardware_df, price_df)

In [7]:
if estimation_method == 'hardware-capex-opex':
    frontier_pcd_df_copy = frontier_pcd_df.copy()
    with open(f'{results_dir}/component_cost_estimation.out', 'w') as f:
        with redirect_stdout(f):
            component_cost_df = cost_estimation_function(frontier_pcd_df_copy, hardware_df, price_df, separate_components=True)

In [8]:
cost_df

,System,Domain,Task,Authors,Notability criteria,Notability criteria notes,Model accessibility,Link,Citations,Reference,...,Foundation model,Training compute lower bound,Training compute upper bound,Training chip-hours,Code accessibility,Dataset accessibility,Accessibility notes,Organization categorization (from Organization),Possibly over 1e23 FLOP,Cost
75,Gemini Ultra,Multimodal,"Language modelling,Visual question answering,C...",Gemini Team,SOTA improvement,""" Evaluation on a broad range of benchmarks sh...",Hosted access (no API),https://storage.googleapis.com/deepmind-media/...,633.0,Gemini: A Family of Highly Capable Multimodal ...,...,NaN,NaN,NaN,132000000.0,NaN,NaN,NaN,Industry,NaN,9.590527e+07
91,Inflection-2,Language,Language modelling,NaN,Significant use,Inflection-2 either already powers Pi or soon ...,Hosted access (no API),https://inflection.ai/inflection-2,NaN,Inflection-2: The Next Step Up,...,checked,NaN,NaN,NaN,NaN,NaN,"via Pi, no API",Industry,NaN,1.824528e+07
116,Grok-1,Language,"Language modelling,Chat",NaN,SOTA improvement,"""On these benchmarks, Grok-1 displayed strong ...",Open source,"https://x.ai/model-card/, https://x.ai/blog/gr...",NaN,Announcing Grok,...,checked,2,7.0,NaN,Unreleased,Unreleased,apache 2.0,Industry,checked,NaN
130,ChatGLM3,Multimodal,"Chat,Visual question answering",NaN,SOTA improvement,"Aiming at GPT-4V, ChatGLM3 has implemented ite...",NaN,https://www.zhipuai.cn/en/news/76,NaN,Zhipu AI launches third-generation base model,...,checked,NaN,NaN,NaN,NaN,NaN,NaN,Industry,NaN,NaN
176,Falcon-180B,Language,Language modelling,"Ebtesam Almazrouei, Hamza Alobeidli, Abdulaziz...",SOTA improvement,"""It's currently at the top of the Hugging Face...",Open access (restricted use),https://falconllm.tii.ae/falcon-180b.html; htt...,86.0,The Falcon Series of Open Language Models,...,NaN,NaN,NaN,17694720.0,NaN,NaN,"""Falcon 180b can be commercially used but unde...",Government,NaN,1.427716e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1145,Xception,Vision,Image classification,François Chollet,Highly cited,NaN,NaN,https://arxiv.org/abs/1610.02357,11578.0,Xception: Deep Learning with Depthwise Separab...,...,NaN,NaN,NaN,43200.0,NaN,NaN,NaN,Industry,NaN,1.601676e+04
1146,GNMT,Language,Translation,"Yonghui Wu, Mike Schuster, Zhifeng Chen, Quoc ...",Highly cited,NaN,Hosted access (no API),https://arxiv.org/abs/1609.08144,6196.0,Google's Neural Machine Translation System: Br...,...,NaN,NaN,NaN,414720.0,NaN,NaN,presumably deployed via Google translate,Industry,NaN,1.753311e+05
1182,AlphaGo Lee,Games,Go,"David Silver, Aja Huang, Chris J. Maddison, Ar...",Highly cited,NaN,NaN,https://www.nature.com/articles/nature16961,14887.0,Mastering the game of Go with deep neural netw...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Industry,NaN,NaN
1186,ResNet-152 (ImageNet),Vision,Image classification,"Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun",Highly cited,NaN,NaN,https://arxiv.org/abs/1512.03385,156882.0,Deep Residual Learning for Image Recognition,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Industry,NaN,NaN


In [9]:
cost_df['Cost'].notna().sum()

47

Use the below to check data availability for specific systems

In [10]:
# system = 'WizardLM-7B'
# row = cost_df.loc[cost_df['System'] == system]
# print('Cost:', row['Cost'].values[0])
# print('Training hardware:', row['Training hardware'].values[0])
# print('Training time (hours):', row['Training time (hours)'].values[0])
# print('Hardware quantity:', row['Hardware quantity'].values[0])
# print('Hardware utilization:', row['Hardware utilization'].values[0])

# Apply inflation adjustment

In [11]:
cost_df['Cost'].dropna()

75      9.590527e+07
91      1.824528e+07
176     1.427716e+07
215     1.535585e+06
227     2.514884e+06
268     1.517401e+07
309     5.752012e+07
325     7.928522e+05
369     6.498636e+06
380     1.237991e+06
395     9.354507e+06
420     2.016533e+06
421     1.123767e+06
442     1.001441e+07
448     1.081508e+06
485     1.014016e+06
495     9.299755e+06
517     5.789905e+05
546     1.700781e+06
550     1.583314e+06
590     5.081185e+06
597     7.986971e+05
617     2.198945e+05
639     2.401743e+05
647     2.229937e+05
684     1.406023e+05
700     3.646119e+05
703     1.597190e+05
765     1.322771e+05
772     2.835394e+06
807     6.958371e+04
812     5.455145e+05
839     3.282400e+05
843     1.969940e+05
857     1.042778e+06
858     1.405011e+05
883     1.088780e+05
969     1.351446e+04
1042    1.083686e+06
1058    1.640608e+06
1092    2.194649e+04
1114    4.877198e+03
1118    1.254874e+06
1134    7.210460e+02
1145    1.601676e+04
1146    1.753311e+05
1188    3.081551e+02
Name: Cost, d

In [12]:
cost_df = adjust_column_for_inflation(cost_df, 'Cost', 'data/PCU518210518210.csv', '2023-12-01')

In [13]:
cost_df['Cost (inflation-adjusted)'].dropna()

75      9.590527e+07
91      1.828799e+07
176     1.436562e+07
215     1.544343e+06
227     2.529227e+06
268     1.533011e+07
309     5.813749e+07
325     8.029780e+05
369     6.629295e+06
380     1.262881e+06
395     9.543076e+06
420     2.056759e+06
421     1.146184e+06
442     1.020343e+07
448     1.101921e+06
485     1.030060e+06
495     9.453757e+06
517     5.985725e+05
546     1.755101e+06
550     1.633883e+06
590     5.241881e+06
597     8.254158e+05
617     2.268665e+05
639     2.483748e+05
647     2.306077e+05
684     1.452769e+05
700     3.783762e+05
703     1.657485e+05
765     1.379925e+05
772     2.976164e+06
807     7.297400e+04
812     5.731032e+05
839     3.497799e+05
843     2.099212e+05
857     1.112203e+06
858     1.498552e+05
883     1.171768e+05
969     1.455771e+04
1042    1.171582e+06
1058    1.773674e+06
1092    2.370500e+04
1114    5.325990e+03
1118    1.370346e+06
1134    7.873954e+02
1145    1.749060e+04
1146    1.914648e+05
1188    3.428010e+02
Name: Cost (i

In [14]:
assert cost_df['Cost (inflation-adjusted)'].notna().sum() == cost_df['Cost'].notna().sum()

# Regression

In [15]:
cost_df['Publication date (float)'] = datetime_to_float_year(pd.to_datetime(cost_df['Publication date']))

In [16]:
reg_results = fit_ols_regression(cost_df, ['Publication date (float)'], 'Cost (inflation-adjusted)', logy=True)
reg_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.580
Model:                            OLS   Adj. R-squared:                  0.571
Method:                 Least Squares   F-statistic:                     62.23
Date:                Fri, 10 May 2024   Prob (F-statistic):           5.01e-10
Time:                        19:10:37   Log-Likelihood:                -51.084
No. Observations:                  47   AIC:                             106.2
Df Residuals:                      45   BIC:                             109.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -753.7811     96.287     -7.828      0.000    -947.714    -559.849
x1             0.3759      0.048      7.889      0.000       0.280       0.472
==============================================================================
Omnibus:                        2.459   Durbin-Watson:                   2.264
Prob(Omnibus):                  0.292   Jarque-Bera (JB):                2.083
Skew:                           0.513   Prob(JB):                        0.353
Kurtosis:                       2.895   Cond. No.                     1.82e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.82e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [17]:
with open(f'{results_dir}/regression_results.out', 'w') as f:
    with redirect_stdout(f):
        print_growth_rates(reg_results)
print_growth_rates(reg_results)

N=47.0
R^2=0.58
0.38 OOMs/year (95% CI: 0.28, 0.47)
2.4x/year (95% CI: 1.9x, 3.0x)
doubling time of 10 months (95% CI: 8, 13)


In [18]:
pred_start_year = 2015
pred_end_year = 2025
pred_start_date = f'{pred_start_year}-01-01'
pred_end_date = f'{pred_end_year}-01-01'

pred_years = pd.DataFrame({'Publication date (float)': np.linspace(pred_start_year, pred_end_year, 100)})
pred_years

,Publication date (float)
0,2015.00000
1,2015.10101
2,2015.20202
3,2015.30303
4,2015.40404
...,...
95,2024.59596
96,2024.69697
97,2024.79798
98,2024.89899


In [19]:

predicted_cost_df = get_predictions(reg_results, pred_years, ['Publication date (float)'])
predicted_cost_df['Publication date'] = predicted_cost_df['Publication date (float)'].apply(float_year_to_datetime)
predicted_cost_df

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper,Publication date (float),Publication date
0,3.587178,0.299129,2.984701,4.189656,1.992222,5.182135,2015.00000,2015-01-01
1,3.625145,0.294640,3.031710,4.218579,2.033582,5.216707,2015.10101,2015-02-06
2,3.663111,0.290160,3.078698,4.247524,2.074890,5.251332,2015.20202,2015-03-15
3,3.701077,0.285692,3.125664,4.276490,2.116146,5.286008,2015.30303,2015-04-21
4,3.739043,0.281235,3.172607,4.305479,2.157349,5.320737,2015.40404,2015-05-28
...,...,...,...,...,...,...,...,...
95,7.193965,0.207539,6.775960,7.611970,5.659158,8.728773,2024.59596,2024-08-06
96,7.231932,0.211678,6.805590,7.658273,5.694832,8.769031,2024.69697,2024-09-12
97,7.269898,0.215845,6.835163,7.704632,5.730450,8.809346,2024.79798,2024-10-19
98,7.307864,0.220038,6.864684,7.751044,5.766009,8.849718,2024.89899,2024-11-25


# Export data

In [20]:
keep_cols = [
    'System',
    'Domain',
    'Task',
    'Model accessibility',
    'Reference',
    'Publication date',
    'Organization',
    'Parameters',
    'Training compute (FLOP)',
    'Training dataset size (datapoints)',
    'Epochs',
    'Training time (hours)',
    'Training hardware',
    'Country (from Organization)',
    'Base model',
    'Finetune compute (FLOP)',
    'Hardware quantity',
    'Hardware utilization',
    'Training cloud compute vendor',
    'Training data center',
    # 'Training time (chip hours)',
    'Cost',
    'Cost (inflation-adjusted)',
]
cost_df[keep_cols]

,System,Domain,Task,Model accessibility,Reference,Publication date,Organization,Parameters,Training compute (FLOP),Training dataset size (datapoints),...,Training hardware,Country (from Organization),Base model,Finetune compute (FLOP),Hardware quantity,Hardware utilization,Training cloud compute vendor,Training data center,Cost,Cost (inflation-adjusted)
75,Gemini Ultra,Multimodal,"Language modelling,Visual question answering,C...",Hosted access (no API),Gemini: A Family of Highly Capable Multimodal ...,2023-12-06,Google DeepMind,NaN,5.000000e+25,NaN,...,Google TPU v4,Multinational,NaN,NaN,55000.0,NaN,NaN,NaN,9.590527e+07,9.590527e+07
91,Inflection-2,Language,Language modelling,Hosted access (no API),Inflection-2: The Next Step Up,2023-11-22,Inflection AI,NaN,1.001000e+25,NaN,...,NVIDIA H100 SXM5,United States of America,NaN,NaN,5000.0,NaN,NaN,NaN,1.824528e+07,1.828799e+07
116,Grok-1,Language,"Language modelling,Chat",Open source,Announcing Grok,2023-11-04,xAI,3.140000e+11,2.900000e+24,NaN,...,NaN,United States of America,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130,ChatGLM3,Multimodal,"Chat,Visual question answering",NaN,Zhipu AI launches third-generation base model,2023-10-27,Zhipu AI,1.300000e+11,1.092000e+24,1.050000e+12,...,NaN,China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
176,Falcon-180B,Language,Language modelling,Open access (restricted use),The Falcon Series of Open Language Models,2023-09-06,Technology Innovation Institute,1.800000e+11,3.760000e+24,2.625000e+12,...,NVIDIA A100 SXM4 40 GB,United Arab Emirates,NaN,NaN,4096.0,0.1876,Amazon Web Services,NaN,1.427716e+07,1.436562e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1145,Xception,Vision,Image classification,NaN,Xception: Deep Learning with Depthwise Separab...,2016-10-07,Google,2.285595e+07,4.360000e+20,3.500000e+08,...,NVIDIA Tesla K80,United States of America,NaN,NaN,60.0,NaN,NaN,NaN,1.601676e+04,1.749060e+04
1146,GNMT,Language,Translation,Hosted access (no API),Google's Neural Machine Translation System: Br...,2016-09-26,Google,2.780000e+08,6.900000e+21,3.600000e+08,...,NVIDIA Tesla K80,United States of America,NaN,NaN,96.0,NaN,NaN,NaN,1.753311e+05,1.914648e+05
1182,AlphaGo Lee,Games,Go,NaN,Mastering the game of Go with deep neural netw...,2016-01-27,DeepMind,NaN,1.900000e+21,2.940000e+07,...,NaN,United Kingdom of Great Britain and Northern I...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1186,ResNet-152 (ImageNet),Vision,Image classification,NaN,Deep Residual Learning for Image Recognition,2015-12-10,Microsoft,6.000000e+07,1.210000e+19,1.280000e+06,...,NaN,United States of America,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
cost_df[keep_cols].to_csv(results_dir + 'price dataset.csv', index=False)

# Plots

In [22]:
fig = px.scatter(
    cost_df,
    x='Publication date',
    y='Cost (inflation-adjusted)',
    text='System',
    log_y=True,
)
fig.update_traces(textposition='top center')

# no legend
fig.update_layout(showlegend=False)

# axis labels
fig.update_xaxes(title_text='Publication date')
fig.update_yaxes(title_text='Cost (2023 USD)')

# title
fig.update_layout(title_text=get_cost_plot_title(estimation_method, compute_threshold_method, compute_threshold))

# update size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
    title_font=dict(
        size=16,
    )
)

# font size
fig.update_layout(
    font=dict(
        size=14,
    )
)

# axis limits
fig.update_xaxes(range=['2015-01-01', '2025-01-01'])

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'cost_scatter')

fig.show()

In [23]:
label_systems = ['GNMT', 'AlphaGo Master', 'AlphaGo Zero', 'AlphaZero', 'DALL-E', 'GPT-3 175B (davinci)', 'PaLM (540B)', 'Llama 2-70B', 'Falcon 180B', 'GPT-4', 'Gemini Ultra', 'Inflection-2']

tpu_mask = cost_df['Training hardware'].str.contains('TPU', na=False)
tpu_cost_df = cost_df.loc[tpu_mask]
gpu_cost_df = cost_df.loc[~tpu_mask]

fig = px.scatter(
    gpu_cost_df,
    x='Publication date',
    y='Cost (inflation-adjusted)',
    log_y=True,
)
fig.add_scatter(
    x=tpu_cost_df['Publication date'],
    y=tpu_cost_df['Cost (inflation-adjusted)'],
    mode='markers',
    marker_symbol='circle-open' if estimation_method != 'cloud' else 'circle',
    name='Using equivalent price of TPU' if estimation_method != 'cloud' else '',
    showlegend=estimation_method != 'cloud',
)
fig.add_scatter(
    x=gpu_cost_df.loc[gpu_cost_df['System'].isin(label_systems)]['Publication date'],
    y=gpu_cost_df.loc[gpu_cost_df['System'].isin(label_systems)]['Cost (inflation-adjusted)'],
    text=gpu_cost_df.loc[gpu_cost_df['System'].isin(label_systems)]['System'],
    mode='text',
    showlegend=False,
)
fig.add_scatter(
    x=tpu_cost_df.loc[tpu_cost_df['System'].isin(label_systems)]['Publication date'],
    y=tpu_cost_df.loc[tpu_cost_df['System'].isin(label_systems)]['Cost (inflation-adjusted)'],
    text=tpu_cost_df.loc[tpu_cost_df['System'].isin(label_systems)]['System'],
    mode='text',
    showlegend=False,
)

# Marker color
fig.update_traces(
    marker=dict(
        color='rgb(0,100,200)',
    ),
    selector=dict(mode='markers'),
)

# Shade in CI
fig.add_scatter(
    x=predicted_cost_df['Publication date'],
    y=10**predicted_cost_df['mean_ci_lower'],
    mode='lines',
    line=dict(width=0),
    showlegend=False,
)
fig.add_scatter(
    x=predicted_cost_df['Publication date'],
    y=10**predicted_cost_df['mean_ci_upper'],
    mode='lines',
    fill='tonexty',
    fillcolor='rgba(0,100,200,0.2)',
    line=dict(width=0),
    name='95% CI of mean',
)
fig.add_scatter(
    x=predicted_cost_df['Publication date'],
    y=10**predicted_cost_df['mean'],
    mode='lines',
    line=dict(color='rgb(0,100,200)'),
    name=f'Regression mean (growth rate: {10**reg_results.params[1]:.1f}x per year)',
)

fig.update_traces(textposition='top center')

# axis limits
fig.update_xaxes(range=['2015-01-01', '2025-01-01'])
if estimation_method == 'hardware-acquisition':
    fig.update_yaxes(range=[4, 10])
else:
    fig.update_yaxes(range=[1, 9])

# legend on bottom-right of the axes
fig.update_layout(
    legend=dict(
        x=0.45,
        y=0.05,
    )
)

# axis labels
fig.update_xaxes(title_text='Publication date')
fig.update_yaxes(title_text='Cost (2023 USD)')

# title
fig.update_layout(title_text=get_cost_plot_title(estimation_method, compute_threshold_method, compute_threshold))

# update size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
    title_font=dict(
        size=16,
    )
)

# font size
fig.update_layout(
    font=dict(
        size=14,
    )
)

# axis limits
fig.update_xaxes(range=[pred_start_date, pred_end_date])

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'cost_regression')

fig.show()

# Cost components

In [24]:
cost_component_names = [
    'AI accelerator chip cost',
    'Other server components cost',
    'Cluster-level interconnect cost',
    'Energy cost',
]

In [25]:
for key in cost_component_names:
    component_cost_df[f"{key} (%)"] = component_cost_df[key] / component_cost_df['Cost'] * 100
component_cost_df['AI accelerator chip cost (%)']

75      51.403372
91      50.923193
116           NaN
130           NaN
176     48.397554
          ...    
1145    48.715399
1146    49.245254
1182          NaN
1186          NaN
1188    40.989018
Name: AI accelerator chip cost (%), Length: 67, dtype: float64

In [26]:
# Stacked bar chart of cost components, using component_cost_df
cost_component_pc_names = [name + ' (%)' for name in cost_component_names]
fig = px.bar(
    component_cost_df.dropna(subset=cost_component_pc_names).sort_values(by='Publication date'),
    x='System',
    y=cost_component_pc_names,
    barmode='stack',
)

# axis labels
fig.update_xaxes(title_text='ML model')
fig.update_yaxes(title_text='% of hardware CapEx + OpEx')
fig.update_layout(
    legend=dict(
        title_text='Cost component',
        x=0.60,
        y=0.05,
    )
)
# limits 0 to 100
fig.update_yaxes(range=[0, 100])

fig.update_yaxes(tickvals=list(range(0, 101, 10)))

# size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
)

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'cost_component_percentage')

fig.show()

In [27]:
# Stacked bar chart of cost components, using component_cost_df
fig = px.bar(
    component_cost_df.dropna(subset=['Cost']).sort_values(by='Publication date'),
    x='System',
    y='Energy cost %',
    barmode='stack',
    # labels='Cost %',
    # text='Energy cost %',
)
# axis labels
fig.update_xaxes(title_text='System')
fig.update_yaxes(title_text='Energy cost (% of Hardware CapEx + OpEx)')
# fig.update_layout(
#     legend=dict(
#         title_text='Cost component',
#         x=0.75,
#         y=0.05,
#     )
# )
# limits 0 to 100
fig.update_yaxes(range=[0, 30])
# size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
)

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'energy_percentage')

fig.show()

ValueError: Value of 'y' is not the name of a column in 'data_frame'. Expected one of ['System', 'Domain', 'Task', 'Authors', 'Notability criteria', 'Notability criteria notes', 'Model accessibility', 'Link', 'Citations', 'Reference', 'Publication date', 'Organization', 'Parameters', 'Parameters notes', 'Training compute (FLOP)', 'Training compute notes', 'Training dataset', 'Training dataset notes', 'Training dataset size (datapoints)', 'Dataset size notes', 'Epochs', 'Inference compute (FLOP)', 'Inference compute notes', 'Training time (hours)', 'Training time notes', 'Training hardware', 'Approach', 'Training compute cost (2020 USD)', 'Compute cost notes', 'Compute sponsor categorization', 'Confidence', 'Abstract', 'Last modified', 'Created By', 'Benchmark data', 'Exclude', 'Country (from Organization)', 'Base model', 'Finetune compute (FLOP)', 'Finetune compute notes', 'Hardware quantity', 'Hardware utilization', 'Training cost trends', 'Training cloud compute vendor', 'Training da

In [ ]:
# Stacked bar chart of cost components, using component_cost_df
component_cost_df = component_cost_df.dropna(subset=['Cost'])
component_cost_df.loc[:, 'Hardware cost'] = [cost['hardware'] for cost in component_cost_df['Cost']]
component_cost_df.loc[:, 'Energy cost'] = [cost['energy'] for cost in component_cost_df['Cost']]
fig = px.scatter(
    component_cost_df.dropna(subset=['Cost']).sort_values(by='Publication date'),
    x='Publication date',
    y='Energy cost',
    text='System',
)
# axis labels
fig.update_xaxes(title_text='System')
fig.update_yaxes(title_text='Energy cost')
# log y
fig.update_yaxes(type='log')
# size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
)

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'energy_cost')

fig.show()

In [ ]:
from energy import energy_price

# Stacked bar chart of cost components, using component_cost_df
component_cost_df = component_cost_df.dropna(subset=['Cost'])
component_cost_df.loc[:, 'Hardware cost'] = [cost['hardware'] for cost in component_cost_df['Cost']]
component_cost_df.loc[:, 'Energy (kWh)'] = [row['Cost']['energy'] / energy_price(row['Publication date'].year) for _, row in component_cost_df.iterrows()]
fig = px.scatter(
    component_cost_df,
    x='Publication date',
    y='Energy (kWh)',
    text='System',
)
# log y
fig.update_yaxes(type='log')
# size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
)

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'energy_kwh')

fig.show()

In [ ]:
component_cost_df.loc[:, 'Power capacity for 3-month training run (kW)'] = [row['Cost']['energy'] / energy_price(row['Publication date'].year) / (HOURS_PER_YEAR/4) / server_TDP_fraction(row['Training hardware']) for _, row in component_cost_df.iterrows()]
fig = px.scatter(
    component_cost_df,
    x='Publication date',
    y='Power capacity for 3-month training run (kW)',
    text='System',
)
# log y
fig.update_yaxes(type='log')
# size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
)

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'power_capacity_kw')

fig.show()

In [ ]:
component_cost_df['Publication date (float)'] = datetime_to_float_year(pd.to_datetime(component_cost_df['Publication date']))

In [ ]:
energy_reg_results = fit_ols_regression(component_cost_df, ['Publication date (float)'], 'Power capacity for 3-month training run (kW)', logy=True)
energy_reg_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.660
Model:                            OLS   Adj. R-squared:                  0.653
Method:                 Least Squares   F-statistic:                     87.41
Date:                Thu, 09 May 2024   Prob (F-statistic):           4.10e-12
Time:                        12:27:03   Log-Likelihood:                -41.068
No. Observations:                  47   AIC:                             86.14
Df Residuals:                      45   BIC:                             89.84
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -725.0174     77.807     -9.318      0.000    -881.728    -568.307
x1             0.3600      0.039      9.349      0.000       0.282       0.438
==============================================================================
Omnibus:                        3.175   Durbin-Watson:                   2.213
Prob(Omnibus):                  0.204   Jarque-Bera (JB):                2.964
Skew:                           0.553   Prob(JB):                        0.227
Kurtosis:                       2.461   Cond. No.                     1.82e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.82e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""